In [ ]:
import pandas as pd
import numpy as np
from p_tqdm import p_map

from emoji import unicode_codes 

In [ ]:
import sys
sys.path.append("..")
from src.data.utils import parallelize_dataframe, find_emojis

In [ ]:
import gzip
path = "/home/jczestochowska/workspace/dlab/emoji-ambiguity/data/external/emojitweets-01-04-2018.txt.gz"
tweets = []
with gzip.open(path, mode='rt', encoding='utf-8', newline='\n') as f:
    for line in f.readlines():
        tweets.append(line.strip())

In [ ]:
len(tweets)

In [ ]:
tweets[:10]

In [ ]:
# emoji_tweets = pd.read_table("/home/jczestochowska/workspace/dlab/emoji-ambiguity/data/external/emojitweets-01-04-2018.txt.gz",\
#                              header=None, lineterminator='\n', encoding='utf-8').tolist()
# emoji_tweets = emoji_tweets.rename({0: "tweet"}, axis=1)
# emoji_tweets.head()

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def get_emojis(tweet):
    emojis = set()
    for emoji in find_emojis(tweet):
        emojis.add(emoji)
    return emojis

In [ ]:
all_emojis = set.union(*p_map(get_emojis, tweets, num_cpus=4))

In [ ]:
pool = Pool(4)
start = time.time()
out = set.union(*pool.map(get_emojis, emoji_tweets))
pool.close()
print(time.time() - start)

In [ ]:
pool.close()

In [ ]:
emojis = set()
for _, row in tqdm(emoji_tweets.iterrows()):
    for emoji in find_emojis(row.tweet):
        emojis.add(emoji)
len(emojis)

In [ ]:
emoji_tweets['emojis'] = parallelize_dataframe(emoji_tweets, apply_find_emojis)

In [ ]:
# Add space between any consecutive emojis so that they are treated as separate tokens

In [ ]:
# pd.set_option('display.max_rows', None)
# emoji_tweets.head(100)

In [ ]:
import spacy
from spacymoji import Emoji

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("emoji", first=True)
# doc = nlp('We are all same 👶👶🏻👶🏼👶🏽👶🏾👶🏿 but different in skin colors!')
doc = nlp("💰🚶🏻‍♀️🚶🏻‍♀️🚶🏻‍♀️💃🏻👠 Let em know 👏🏾")

tokenized = [token.text for token in doc]
processed = " ".join(tokenized)
processed

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# s ="💰🚶🏻‍♀️🚶🏻‍♀️🚶🏻‍♀️💃🏻👠 Let em know 👏🏾"
# tokenizer.add_tokens(["💰", "🚶🏻‍♀️", "💃", "👠", "👏🏾"])
print(tokenizer.tokenize(processed))

In [ ]:
################

In [ ]:
def find_emoji_indices(row):
    out, emoji_token = [], '[UNK]'
    tokenized = np.array(tokenizer.tokenize(row.tweet))
    emojis = find_emojis(row.tweet)
    ii = np.where(tokenized == emoji_token)[0]
    return list(zip(emojis, ii))

In [ ]:
find_emoji_indices(emoji_tweets.iloc[4])

In [ ]:
emoji_tweets.iloc[3].tweet

In [ ]:
# row = "2️⃣4️⃣ hours 'til our schedule drops!"
row = 'I CANT BREATIUHW 💀💀💀'

In [ ]:
find_emojis(row)

In [ ]:
find_emojis(row)

In [ ]:
row.index(find_emojis(row)[2])

In [ ]:
row[17]

In [ ]:
row[0]

In [ ]:
row = "2️⃣4️⃣ hours 'til our schedule drops!"
# tokenizer.tokenize(row)
# emojis = find_emojis(row)
# emojis
# emojis = find_emojis(row.tweet)
# ii = np.where(tokenized == emoji_token)[0]
# return list(zip(emojis, ii))

In [ ]:
row = "2️⃣4️⃣ hours  'til  our  schedule  drops!"
tokenizer.tokenize(row)

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")

In [ ]:
text = "Squad arriving for Game 2 🚀"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
tokenized_text = tokenizer.tokenize(text)
tokenized_text

In [ ]:
tokenized_text = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [ ]:
output[0][:, 6]